In [1]:
import numpy as np
import scipy as sci
import matplotlib.pyplot as plt
from jinja2 import Template
from pyne import data
from pyne.material import Material
from pyne import serpent

/home/zoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  """
/home/zoe/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)
/home/zoe/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: QAWarning: pyne.material is not yet QA compliant.
  return f(*args, **kwds)
/home/zoe/.local/lib/python3.7/site-packages/pyne/serpent.py:11: QAWarning: pyne.serpent is not yet QA compliant.
  warn(__name__ + " is not yet QA compliant.", QAWarning)


First we want the helium density in $10^24$ atoms per cc.

\begin{align}
PV&=nRT\\
\frac{n}{V}&=\frac{P}{RT}
\end{align}

In [2]:

R = 8.314 # j/k-mol
temp = [250 ,750] #celsius
pres = [6*10**6, 5.84*10**6] #pascals
hedensity = np.zeros(len(temp))


for i, t in enumerate(temp):
    hedensity[i] = ((pres[i])/((t+273.15)*R)) # mol/m^3 

hedensavg = np.sum(hedensity)/len(hedensity)*(6.022*10**-7)# 10^24/cc

print(hedensavg)

0.000622077124202638


### Isotopic composition of the fuel

The enrichment of the UCO kernels in the TRISO particles is known.  Additionally, we know the molar contentrations of the three compounds that make UCO: $UO_2$ , $UC$, and $UC_{1.86}$. The [PyNE](https://pyne.io/) python library is used to create the material cards for the fuel.
<br>
UCO is created from $UO_2$, $UC$, and $UC_{1.86}$ in molar concentrations of 0.714, 0.164, and 0.123, respectively.

In [3]:
leu = Material({'U235': 0.155 , 'U238': 0.845})
uo2 = Material()
uc = Material()
uc186 = Material()

uo2.from_atom_frac({'O16': 2.0 , leu :1.0})
uc.from_atom_frac({'C12':1, leu: 1.0})
uc186.from_atom_frac({'C12':1.86, leu : 1.0})

uco = Material()
uco.from_atom_frac({uo2: 0.714, uc: 0.164, uc186: 0.123})
print(uco.mcnp()) #the negative values for composition mean it is mass fraction, just like serpent


/home/zoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  """Entry point for launching an IPython kernel.


m?
     6012 -1.7761e-02
     8016 -8.6071e-02
     92235 -1.3891e-01
     92238 -7.5726e-01



In order to find the core dimensions, I took an image of the reactor core given by X-energy in a presentation to the NRC.  
<br>
Then, I opened it in an image viewer that gave the cursor's coordinates in pixels, measured the reactor core features in pixels.  Knowing the outer core diameter, developed a scale between pixels and meters, and used that to determine the core geometry.

\begin{align}
s (\frac{cm}{px}) = 100* \frac{RPV_D (m)}{RPV_D (px)}\\
x (cm) = x (px) * scale
\end{align}

In [4]:
core_dm = 4.88 # m
core_dpx = 69 #px
scale = core_dm/core_dpx *100 # cm/px

inner_corerad_px = 17.5
inner_corerad_cm = inner_corerad_px *scale
reflect_px = 13
reflect_cm = reflect_px *scale

print(2*(reflect_cm+inner_corerad_cm)) #can't be more than 488, which is the diameter including RPV
print()
print(inner_corerad_cm)
print(reflect_cm)
print(inner_corerad_cm+reflect_cm)

431.4202898550724

123.76811594202897
91.94202898550724
215.7101449275362


I found the midpoint of the cones on the top and bottom , and used the distance between these two midpoints as an adjusted height.  Then, I simplified by letting the reflector directly surround this region.

In [5]:
mid_bott_y = 228
mid_top_y = 91.5

adj_h_px = mid_bott_y - mid_top_y #px

adj_h_cm = adj_h_px*scale

print(adj_h_cm)

965.391304347826


### Creating the input file

The first step to creating the input file is to create the pebble and particle location input cards.  This can be done using serpent's built in particle dispersal routine, which takes the dimensions of the particle, the volume they are dispersed in, and the particle number or packing fraction, the universe the particles are in, and creates a plain text file, where each line is:

\begin{align}
x-coordinate\ \ y-coordinate\ \  z-coordinate\ \  universe
\end{align}

In order to split the automatically generated distribution file into several sub-files for modeling different fuel compositions, the cell below splits a given text file into multiple.  It can also append a character to the universe name, if needed.

In [6]:
origin = open('test.txt','r')

filenames = ['test1.txt','test2.txt','test3.txt','test4.txt']

lines = 0
for line in list(origin):
    lines += 1
origin.seek(0)
i = 0

file = ['f1','f2','f3','f4']
for index, name in enumerate(filenames):
    file[index] = open(name,'a+')
subuniverse = ['a','b','c','d']    
    
while i < lines:
    for index, f in enumerate(file):
        #use lines below if you need to append a new character to universe name
        #newline = origin.readline()
        #f.write(newline[:-1] + subuniverse[index] + newline[-1:])
        f.write(origin.readline())
        i += 1
        if i == lines:
            break
for f in file:
    f.close()
origin.close()

In [9]:
serpent.parse_dep('../single-pebble/xe-100-passes.inp_dep.m')

ValueError: operands could not be broadcast together with shapes (7,) (0,) 

In [10]:
serpent.parse_res('../single-pebble/xe-100-passes.inp_res.m')

{'np': <module 'numpy' from '/home/zoe/anaconda3/lib/python3.7/site-packages/numpy/__init__.py'>,
 'INPUT_FILE_NAME': array([b'xe-100-passes.inp', b'xe-100-passes.inp', b'xe-100-passes.inp',
        b'xe-100-passes.inp', b'xe-100-passes.inp', b'xe-100-passes.inp',
        b'xe-100-passes.inp'], dtype='|S17'),
 'BALA_SRC_NEUTRON_NXN': array([[  0.   , 123.001],
        [  0.   , 106.924],
        [  0.   , 104.785],
        [  0.   , 100.702],
        [  0.   , 108.093],
        [  0.   , 101.108],
        [  0.   , 107.198]]),
 'INF_CAPT': array([[0.0003569 , 0.00108   , 0.00051415, 0.0004    ],
        [0.00035897, 0.00101   , 0.00063142, 0.00035   ],
        [0.00036081, 0.00119   , 0.00064604, 0.00047   ],
        [0.00036406, 0.00094   , 0.00065786, 0.00041   ],
        [0.00036872, 0.00106   , 0.00066852, 0.00037   ],
        [0.00037187, 0.00098   , 0.00067651, 0.00039   ],
        [0.00037717, 0.00099   , 0.00068312, 0.00049   ]]),
 'TOT_ACTIVITY': array([1.60143e+05, 1.42103e+1